In [1]:
#Basic Tools
import os, sys, glob
import pandas as pd
import numpy as np
import random
import pickle 

#import logistic regression tools
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

#import other classifier tools
from sklearn.metrics import confusion_matrix, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
#Directory of radiomics results
classification_dir = "./results/resampled_radiomics/Final_ResampledClassificationData.csv"
SelectedFeat_dir = "./results/resampled_radiomics/LASSO_SelectedFeatures.csv"
outdir = "./classification/model_selection"
#modeldir = "/Volumes/harmon-lab/liver/classification/classifiers"

In [3]:
#Set Data
classify_df = pd.read_csv(classification_dir)
features = pd.read_csv(SelectedFeat_dir)

# print("Classification Header: ")
# print(classify_df.columns.values)
# print("\nFeature Header: ")
# print(features.columns.values)

In [4]:
ft = features["Feature"].values.tolist()
num_ft = len(ft)

ft = ["File", "NE_Score", "NE_Status", "NE_Class", "Group"] + ft
print(ft)

['File', 'NE_Score', 'NE_Status', 'NE_Class', 'Group', 'original_shape_MajorAxisLength', 'original_shape_Maximum2DDiameterRow', 'original_shape_MeshVolume', 'original_shape_MinorAxisLength', 'original_shape_SurfaceArea', 'original_shape_VoxelVolume', 'original_firstorder_Energy', 'original_firstorder_TotalEnergy', 'original_firstorder_Variance', 'original_gldm_DependenceNonUniformity', 'original_gldm_GrayLevelNonUniformity', 'original_gldm_LargeDependenceEmphasis', 'original_gldm_LargeDependenceHighGrayLevelEmphasis', 'original_glrlm_GrayLevelNonUniformity', 'original_glrlm_RunLengthNonUniformity', 'original_glszm_GrayLevelNonUniformity', 'original_glszm_LargeAreaEmphasis', 'original_glszm_LargeAreaHighGrayLevelEmphasis', 'original_glszm_LargeAreaLowGrayLevelEmphasis', 'original_glszm_ZoneVariance']


In [5]:
feat_data = classify_df[ft]
print(feat_data) 
print(feat_data.columns[-num_ft:].values)

                                                  File  NE_Score NE_Status   
0                   AIR-SCLC-001_20161012_2_CAP.nii.gz  0.129586  positive  \
1                  AIR-SCLC-001_20161012_3_SOFT.nii.gz  0.129586  positive   
2                   AIR-SCLC-002_20160404_5_CAP.nii.gz  0.369555  positive   
3                  AIR-SCLC-002_20160404_6_SOFT.nii.gz  0.369555  positive   
4                   AIR-SCLC-003_20160524_2_CAP.nii.gz  0.238842  positive   
..                                                 ...       ...       ...   
222  AIR-SCLC-085_20200603_5_2X1_STND_AiCE_2.0_CE.n... -0.080472  negative   
223  AIR-SCLC-086_20200527_4_5X5STND_AiCE_5.0_CE.ni...  0.271515  positive   
224  AIR-SCLC-086_20200527_5_2X1_STND_AiCE_2.0_CE.n...  0.271515  positive   
225  AIR-SCLC-086_20200708_3_5X5STND_AiCE_5.0__5X5S... -0.015790  negative   
226  AIR-SCLC-086_20200708_4_2X1_STND_AiCE_2.0__2X1... -0.015790  negative   

     NE_Class  Group  original_shape_MajorAxisLength   
0      

In [6]:
#Get Group DataFrames
train_df = feat_data[feat_data["Group"] == "train"]
val_df = feat_data[feat_data["Group"] == "val"]
test_df = feat_data[feat_data["Group"] == "test"]
merge_train = feat_data[feat_data["Group"] != "test"]

# Training Data

In [7]:
#Get X and Y data
X_train = train_df[train_df.columns[-num_ft:].values]
Y_train = train_df[train_df.columns[3]]

X_val = val_df[val_df.columns[-num_ft:].values]
Y_val = val_df[val_df.columns[3]]

X_test = test_df[test_df.columns[-num_ft:].values]
Y_test = test_df[test_df.columns[3]]
Y_names = test_df["File"].tolist() 

#X_merge = merge_train[merge_train.columns[-num_ft:].values]
#Y_merge = merge_train[merge_train.columns[3]]

## Logistic Regression Classifier (Unweighted)

In [36]:
## Logistic Classifier
clf_log1 = LogisticRegression(random_state=2024, C = 0.5, solver = 'newton-cg',  max_iter = 1000).fit(X_train, Y_train)

#Validation Metrics
ypred_val = clf_log1.predict(X_val)
acc = accuracy_score(Y_val, ypred_val)
sens = recall_score(Y_val, ypred_val)
ppv = precision_score(Y_val, ypred_val)
f1 = f1_score(Y_val, ypred_val)
CM = confusion_matrix(Y_val, ypred_val)
tn, fp, fn, tp = confusion_matrix(Y_val, ypred_val).ravel()
spec = tn / (tn+fp)

print("Validation Results:")
print(ypred_val)

print("\nValidation Metrics: ")
print("Accuracy: " + str(acc))
print("PPV: " + str(ppv))
print("Sensitivity: " + str(sens))
print("Specificity: " + str(spec))
print("F1: " + str (f1))
print("Confusion Matrix: ")
print(CM)

#Test Metrics 
ypred_test = clf_log1.predict(X_test)
acc = accuracy_score(Y_test, ypred_test)
sens = recall_score(Y_test, ypred_test)
ppv = precision_score(Y_test, ypred_test)
f1 = f1_score(Y_test, ypred_test)
CM = confusion_matrix(Y_test, ypred_test)
tn, fp, fn, tp = confusion_matrix(Y_test, ypred_test).ravel()
spec = tn / (tn+fp)

print("\n\nTesting Results:")
print(ypred_test)

print("\nTesting Metrics: ")
print("Accuracy: " + str(acc))
print("PPV: " + str(ppv))
print("Sensitivity: " + str(sens))
print("Specificity: " + str(spec))
print("F1: " + str (f1))
print("Confusion Matrix: ")
print(CM)

Validation Results:
[1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]

Validation Metrics: 
Accuracy: 0.625
PPV: 0.6451612903225806
Sensitivity: 0.9523809523809523
Specificity: 0.0
F1: 0.7692307692307692
Confusion Matrix: 
[[ 0 11]
 [ 1 20]]


Testing Results:
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]

Testing Metrics: 
Accuracy: 0.8
PPV: 0.8043478260869565
Sensitivity: 0.9736842105263158
Specificity: 0.25
F1: 0.8809523809523809
Confusion Matrix: 
[[ 3  9]
 [ 1 37]]


/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


## Random Forest (Unweighted)

In [48]:
## Random Forest Classifier
clf_rfc1 = RandomForestClassifier(n_estimators = 1000, random_state=200, criterion = "gini", min_samples_leaf = 2).fit(X_train, Y_train)

#Validation Metrics
ypred_val = clf_rfc1.predict(X_val)
acc = accuracy_score(Y_val, ypred_val)
sens = recall_score(Y_val, ypred_val)
ppv = precision_score(Y_val, ypred_val)
f1 = f1_score(Y_val, ypred_val)
CM = confusion_matrix(Y_val, ypred_val)
tn, fp, fn, tp = confusion_matrix(Y_val, ypred_val).ravel()
spec = tn / (tn+fp)

print("Validation Results:")
print(ypred_val)

print("\nValidation Metrics: ")
print("Accuracy: " + str(acc))
print("PPV: " + str(ppv))
print("Sensitivity: " + str(sens))
print("Specificity: " + str(spec))
print("F1: " + str (f1))
print("Confusion Matrix: ")
print(CM)

#Test Metrics 
ypred_test = clf_rfc1.predict(X_test)
acc = accuracy_score(Y_test, ypred_test)
sens = recall_score(Y_test, ypred_test)
ppv = precision_score(Y_test, ypred_test)
f1 = f1_score(Y_test, ypred_test)
CM = confusion_matrix(Y_test, ypred_test)
tn, fp, fn, tp = confusion_matrix(Y_test, ypred_test).ravel()
spec = tn / (tn+fp)

print("\n\nTesting Results:")
print(ypred_test)

print("\nTesting Metrics: ")
print("Accuracy: " + str(acc))
print("PPV: " + str(ppv))
print("Sensitivity: " + str(sens))
print("Specificity: " + str(spec))
print("F1: " + str (f1))
print("Confusion Matrix: ")
print(CM)

Validation Results:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1]

Validation Metrics: 
Accuracy: 0.59375
PPV: 0.6333333333333333
Sensitivity: 0.9047619047619048
Specificity: 0.0
F1: 0.7450980392156863
Confusion Matrix: 
[[ 0 11]
 [ 2 19]]


Testing Results:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 0 1 1 0 0 0 0]

Testing Metrics: 
Accuracy: 0.64
PPV: 0.7380952380952381
Sensitivity: 0.8157894736842105
Specificity: 0.08333333333333333
F1: 0.775
Confusion Matrix: 
[[ 1 11]
 [ 7 31]]


## XGBoost Classifier (Unweighted)

In [50]:
## XGBoost Classifier
clf_xgb1 = xgb.XGBClassifier(tree_method="hist", seed = 100, booster = 'gblinear').fit(X_train, Y_train)

#Validation Metrics
ypred_val = clf_xgb1.predict(X_val)
acc = accuracy_score(Y_val, ypred_val)
sens = recall_score(Y_val, ypred_val)
ppv = precision_score(Y_val, ypred_val)
f1 = f1_score(Y_val, ypred_val)
CM = confusion_matrix(Y_val, ypred_val)
tn, fp, fn, tp = confusion_matrix(Y_val, ypred_val).ravel()
spec = tn / (tn+fp)

print("Validation Results:")
print(ypred_val)

print("\nValidation Metrics: ")
print("Accuracy: " + str(acc))
print("PPV: " + str(ppv))
print("Sensitivity: " + str(sens))
print("Specificity: " + str(spec))
print("F1: " + str (f1))
print("Confusion Matrix: ")
print(CM)

#Test Metrics 
ypred_test = clf_xgb1.predict(X_test)
acc = accuracy_score(Y_test, ypred_test)
sens = recall_score(Y_test, ypred_test)
ppv = precision_score(Y_test, ypred_test)
f1 = f1_score(Y_test, ypred_test)
CM = confusion_matrix(Y_test, ypred_test)
tn, fp, fn, tp = confusion_matrix(Y_test, ypred_test).ravel()
spec = tn / (tn+fp)

print("\n\nTesting Results:")
print(ypred_test)

print("\nTesting Metrics: ")
print("Accuracy: " + str(acc))
print("PPV: " + str(ppv))
print("Sensitivity: " + str(sens))
print("Specificity: " + str(spec))
print("F1: " + str (f1))
print("Confusion Matrix: ")
print(CM)

Validation Results:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]

Validation Metrics: 
Accuracy: 0.65625
PPV: 0.65625
Sensitivity: 1.0
Specificity: 0.0
F1: 0.7924528301886793
Confusion Matrix: 
[[ 0 11]
 [ 0 21]]


Testing Results:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 0 0 0 0 0]

Testing Metrics: 
Accuracy: 0.66
PPV: 0.7560975609756098
Sensitivity: 0.8157894736842105
Specificity: 0.16666666666666666
F1: 0.7848101265822786
Confusion Matrix: 
[[ 2 10]
 [ 7 31]]


/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [15:55:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)


# Weighted Training

In [51]:
## Check Data Distribution
Y_testNE = len (test_df[test_df["NE_Class"] == 1])
Y_testNonNE = len(test_df[test_df["NE_Class"] == 0])
ytest_frac = Y_testNE / Y_testNonNE

Y_mergeNE = len(merge_train[merge_train["NE_Class"] == 1])
Y_mergeNonNE = len(merge_train[merge_train["NE_Class"] == 0])
ymerge_frac = Y_mergeNE / Y_mergeNonNE

Y_allNE = len(classify_df[classify_df["NE_Class"] == 1])
Y_allNonNE = len(classify_df[classify_df["NE_Class"] == 0])
y_all_frac = Y_allNE / Y_allNonNE

print(ytest_frac)
print(ymerge_frac)
print(y_all_frac)

#Do the inverse because of flipping operation (XGBoost)
inv_yall_frac = 1/y_all_frac

print("\nAll Data Distribution:")
print(inv_yall_frac)

print("\nTraining Data Distribution")
inv_ytrain_frac = 1/ymerge_frac
print(inv_ytrain_frac)

3.1666666666666665
3.116279069767442
3.1272727272727274

All Data Distribution:
0.3197674418604651

Training Data Distribution
0.3208955223880597


In [119]:
increment = np.arange(1, 10.25, 0.25)
pos_class = increment * inv_ytrain_frac
neg_class = increment * (1-inv_ytrain_frac)

temp_scale = {"class_weight":[], "neg_scale":[], "pos_scale":[]}

for n, neg in enumerate(neg_class): 
    for p, pos in enumerate(pos_class): 
        temp_scale["class_weight"].append([{0:neg, 1:pos}])
        temp_scale["neg_scale"].append(neg/(1-inv_ytrain_frac))
        temp_scale["pos_scale"].append(pos/inv_ytrain_frac)

In [120]:
print(temp_scale['class_weight'])
scale_df = pd.DataFrame(temp_scale)
scale_df.to_csv(os.path.join(outdir, "ScalingQuantities.csv"), index = False)

[[{0: 0.6791044776119404, 1: 0.3208955223880597}], [{0: 0.6791044776119404, 1: 0.4011194029850746}], [{0: 0.6791044776119404, 1: 0.48134328358208955}], [{0: 0.6791044776119404, 1: 0.5615671641791045}], [{0: 0.6791044776119404, 1: 0.6417910447761194}], [{0: 0.6791044776119404, 1: 0.7220149253731343}], [{0: 0.6791044776119404, 1: 0.8022388059701492}], [{0: 0.6791044776119404, 1: 0.8824626865671641}], [{0: 0.6791044776119404, 1: 0.9626865671641791}], [{0: 0.6791044776119404, 1: 1.042910447761194}], [{0: 0.6791044776119404, 1: 1.123134328358209}], [{0: 0.6791044776119404, 1: 1.2033582089552237}], [{0: 0.6791044776119404, 1: 1.2835820895522387}], [{0: 0.6791044776119404, 1: 1.3638059701492538}], [{0: 0.6791044776119404, 1: 1.4440298507462686}], [{0: 0.6791044776119404, 1: 1.5242537313432836}], [{0: 0.6791044776119404, 1: 1.6044776119402984}], [{0: 0.6791044776119404, 1: 1.6847014925373134}], [{0: 0.6791044776119404, 1: 1.7649253731343282}], [{0: 0.6791044776119404, 1: 1.8451492537313432}], 

## Weighted Logistic Regression

In [112]:
# initialize model
lr_weights = [inv_ytrain_frac]
val_dict = {"Params":[], "Accuracy":[], "PPV":[], "Sensitivity":[], "Specificity":[], "F1_Score":[], "CM":[], "Pred_Value":[]}
test_dict = {"Params":[], "Accuracy":[], "PPV":[], "Sensitivity":[], "Specificity":[], "F1_Score":[], "CM":[], "Pred_Value":[]}

#iterate params search
for i, pair in enumerate(temp_scale['class_weight']):
    param_grid = {'class_weight': temp_scale['class_weight'][i]}
    #print(param_grid)

    for key, space in param_grid.items():
        print("-"*50)
        print("\nWeighing Scheme for Training: ")
        value = param_grid["class_weight"][0]
        print(value)

        clf_log2 = LogisticRegression(random_state=2024, C = 0.5, solver = 'newton-cg',  max_iter = 1000, class_weight = value).fit(X_train, Y_train)

        #Validation Metrics
        ypred_val = clf_log2.predict(X_val)
        acc = accuracy_score(Y_val, ypred_val)
        sens = recall_score(Y_val, ypred_val)
        ppv = precision_score(Y_val, ypred_val)
        f1 = f1_score(Y_val, ypred_val)
        CM = confusion_matrix(Y_val, ypred_val)
        tn, fp, fn, tp = confusion_matrix(Y_val, ypred_val).ravel()
        spec = tn / (tn+fp)

        print("Saving Validation Results:")
        val_dict["Params"].append(value)
        val_dict["Accuracy"].append(acc)
        val_dict["PPV"].append(ppv)
        val_dict["Sensitivity"].append(sens)
        val_dict["Specificity"].append(spec)
        val_dict["F1_Score"].append(f1)
        val_dict["CM"].append(CM)
        val_dict["Pred_Value"].append(ypred_val)
        
        #Test Metrics 
        ypred_test = clf_log2.predict(X_test)
        acc = accuracy_score(Y_test, ypred_test)
        sens = recall_score(Y_test, ypred_test)
        ppv = precision_score(Y_test, ypred_test)
        f1 = f1_score(Y_test, ypred_test)
        CM = confusion_matrix(Y_test, ypred_test)
        tn, fp, fn, tp = confusion_matrix(Y_test, ypred_test).ravel()
        spec = tn / (tn+fp)

        print("Saving Test Results:")
        test_dict["Params"].append(value)
        test_dict["Accuracy"].append(acc)
        test_dict["PPV"].append(ppv)
        test_dict["Sensitivity"].append(sens)
        test_dict["Specificity"].append(spec)
        test_dict["F1_Score"].append(f1)
        test_dict["CM"].append(CM)
        test_dict["Pred_Value"].append(ypred_test)

        

--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.3208955223880597}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.4011194029850746}


/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did no

Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.48134328358208955}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.5615671641791045}


/Users/tys2/anaconda3/lib/python3.10/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/tys2/anaconda3/lib/python3.10

Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.6417910447761194}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.7220149253731343}


...

/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did no

Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 2.968283582089552}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 3.048507462686567}


/Users/tys2/anaconda3/lib/python3.10/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/tys2/anaconda3/lib/python3.10

Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 3.1287313432835817}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 3.2089552238805967}
Saving Validation Results:
Saving Test Results:


/Users/tys2/anaconda3/lib/python3.10/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


In [116]:
#save validation results
val_results = pd.DataFrame(val_dict)
val_results.to_csv(os.path.join(outdir, "LogisticRegression_ValResults.csv"), index = False)


#save test results
test_results = pd.DataFrame(test_dict)
test_results.to_csv(os.path.join(outdir, "LogisticRegression_TestResults.csv"), index = False)

## Random Forest

In [121]:
# initialize model
lr_weights = [inv_ytrain_frac]
val_dict = {"Params":[], "Accuracy":[], "PPV":[], "Sensitivity":[], "Specificity":[], "F1_Score":[], "CM":[], "Pred_Value":[]}
test_dict = {"Params":[], "Accuracy":[], "PPV":[], "Sensitivity":[], "Specificity":[], "F1_Score":[], "CM":[], "Pred_Value":[]}

#iterate params search
for i, pair in enumerate(temp_scale['class_weight']):
    param_grid = {'class_weight': temp_scale['class_weight'][i]}
    #print(param_grid)

    for key, space in param_grid.items():
        print("-"*50)
        print("\nWeighing Scheme for Training: ")
        value = param_grid["class_weight"][0]
        print(value)

        clf = RandomForestClassifier(n_estimators = 1000, random_state=200, criterion = "gini", min_samples_leaf = 2, class_weight = value).fit(X_train, Y_train)

        #Validation Metrics
        ypred_val = clf.predict(X_val)
        acc = accuracy_score(Y_val, ypred_val)
        sens = recall_score(Y_val, ypred_val)
        ppv = precision_score(Y_val, ypred_val)
        f1 = f1_score(Y_val, ypred_val)
        CM = confusion_matrix(Y_val, ypred_val)
        tn, fp, fn, tp = confusion_matrix(Y_val, ypred_val).ravel()
        spec = tn / (tn+fp)

        print("Saving Validation Results:")
        val_dict["Params"].append(value)
        val_dict["Accuracy"].append(acc)
        val_dict["PPV"].append(ppv)
        val_dict["Sensitivity"].append(sens)
        val_dict["Specificity"].append(spec)
        val_dict["F1_Score"].append(f1)
        val_dict["CM"].append(CM)
        val_dict["Pred_Value"].append(ypred_val)
        
        #Test Metrics 
        ypred_test = clf.predict(X_test)
        acc = accuracy_score(Y_test, ypred_test)
        sens = recall_score(Y_test, ypred_test)
        ppv = precision_score(Y_test, ypred_test)
        f1 = f1_score(Y_test, ypred_test)
        CM = confusion_matrix(Y_test, ypred_test)
        tn, fp, fn, tp = confusion_matrix(Y_test, ypred_test).ravel()
        spec = tn / (tn+fp)

        print("Saving Test Results:")
        test_dict["Params"].append(value)
        test_dict["Accuracy"].append(acc)
        test_dict["PPV"].append(ppv)
        test_dict["Sensitivity"].append(sens)
        test_dict["Specificity"].append(spec)
        test_dict["F1_Score"].append(f1)
        test_dict["CM"].append(CM)
        test_dict["Pred_Value"].append(ypred_test)

        

--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.3208955223880597}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.4011194029850746}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.48134328358208955}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.5615671641791045}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.6417910447761194}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1:

Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.8488805970149255, 1: 1.123134328358209}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.8488805970149255, 1: 1.2033582089552237}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.8488805970149255, 1: 1.2835820895522387}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.8488805970149255, 1: 1.3638059701492538}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.8488805970149255, 1: 1.4440298507462686}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Sc

Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 1.0186567164179106, 1: 1.9253731343283582}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 1.0186567164179106, 1: 2.0055970149253732}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 1.0186567164179106, 1: 2.085820895522388}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 1.0186567164179106, 1: 2.166044776119403}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 1.0186567164179106, 1: 2.246268656716418}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Sche

Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 1.1884328358208958, 1: 2.7276119402985075}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 1.1884328358208958, 1: 2.807835820895522}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 1.1884328358208958, 1: 2.888059701492537}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 1.1884328358208958, 1: 2.968283582089552}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 1.1884328358208958, 1: 3.048507462686567}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Schem

Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 1.5279850746268657, 1: 0.5615671641791045}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 1.5279850746268657, 1: 0.6417910447761194}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 1.5279850746268657, 1: 0.7220149253731343}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 1.5279850746268657, 1: 0.8022388059701492}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 1.5279850746268657, 1: 0.8824626865671641}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing S

...

Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 5.942164179104478, 1: 2.5671641791044775}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 5.942164179104478, 1: 2.6473880597014925}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 5.942164179104478, 1: 2.7276119402985075}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 5.942164179104478, 1: 2.807835820895522}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 5.942164179104478, 1: 2.888059701492537}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme f

Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.281716417910449, 1: 0.4011194029850746}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.281716417910449, 1: 0.48134328358208955}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.281716417910449, 1: 0.5615671641791045}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.281716417910449, 1: 0.6417910447761194}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.281716417910449, 1: 0.7220149253731343}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Schem

Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.451492537313434, 1: 1.2033582089552237}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.451492537313434, 1: 1.2835820895522387}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.451492537313434, 1: 1.3638059701492538}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.451492537313434, 1: 1.4440298507462686}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.451492537313434, 1: 1.5242537313432836}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme

Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.621268656716419, 1: 2.0055970149253732}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.621268656716419, 1: 2.085820895522388}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.621268656716419, 1: 2.166044776119403}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.621268656716419, 1: 2.246268656716418}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.621268656716419, 1: 2.326492537313433}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for

Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 2.807835820895522}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 2.888059701492537}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 2.968283582089552}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 3.048507462686567}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 3.1287313432835817}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for

In [123]:
#save validation results
val_results = pd.DataFrame(val_dict)
val_results.to_csv(os.path.join(outdir, "RandomForest_ValResults.csv"), index = False)


#save test results
test_results = pd.DataFrame(test_dict)
test_results.to_csv(os.path.join(outdir, "RandomForest_TestResults.csv"), index = False)

## Xgboost

In [127]:
# initialize model
lr_weights = [inv_ytrain_frac]
val_dict = {"Params":[], "Accuracy":[], "PPV":[], "Sensitivity":[], "Specificity":[], "F1_Score":[], "CM":[], "Pred_Value":[], "Fraction":[]}
test_dict = {"Params":[], "Accuracy":[], "PPV":[], "Sensitivity":[], "Specificity":[], "F1_Score":[], "CM":[], "Pred_Value":[], "Fraction":[]}

#iterate params search
for i, pair in enumerate(temp_scale['class_weight']):
    param_grid = {'class_weight': temp_scale['class_weight'][i]}
    #print(param_grid)

    for key, space in param_grid.items():
        print("-"*50)
        print("\nWeighing Scheme for Training: ")
        value = param_grid["class_weight"][0]
        print(value)
        frac = value[0] / value[1]
        

        clf = xgb.XGBClassifier(tree_method="hist", seed = 100, booster = 'gblinear', scale_pos_weight = frac).fit(X_train, Y_train)

        #Validation Metrics
        ypred_val = clf.predict(X_val)
        acc = accuracy_score(Y_val, ypred_val)
        sens = recall_score(Y_val, ypred_val)
        ppv = precision_score(Y_val, ypred_val)
        f1 = f1_score(Y_val, ypred_val)
        CM = confusion_matrix(Y_val, ypred_val)
        tn, fp, fn, tp = confusion_matrix(Y_val, ypred_val).ravel()
        spec = tn / (tn+fp)

        print("Saving Validation Results:")
        val_dict["Params"].append(value)
        val_dict["Accuracy"].append(acc)
        val_dict["PPV"].append(ppv)
        val_dict["Sensitivity"].append(sens)
        val_dict["Specificity"].append(spec)
        val_dict["F1_Score"].append(f1)
        val_dict["CM"].append(CM)
        val_dict["Pred_Value"].append(ypred_val)
        val_dict["Fraction"].append(frac)
        
        #Test Metrics 
        ypred_test = clf.predict(X_test)
        acc = accuracy_score(Y_test, ypred_test)
        sens = recall_score(Y_test, ypred_test)
        ppv = precision_score(Y_test, ypred_test)
        f1 = f1_score(Y_test, ypred_test)
        CM = confusion_matrix(Y_test, ypred_test)
        tn, fp, fn, tp = confusion_matrix(Y_test, ypred_test).ravel()
        spec = tn / (tn+fp)

        print("Saving Test Results:")
        test_dict["Params"].append(value)
        test_dict["Accuracy"].append(acc)
        test_dict["PPV"].append(ppv)
        test_dict["Sensitivity"].append(sens)
        test_dict["Specificity"].append(spec)
        test_dict["F1_Score"].append(f1)
        test_dict["CM"].append(CM)
        test_dict["Pred_Value"].append(ypred_test)
        test_dict["Fraction"].append(frac)

        

--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.3208955223880597}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.4011194029850746}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.48134328358208955}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.5615671641791045}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.6417910447761194}


/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:48:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:48:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:48:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:48:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tys2/anaconda3/lib/python3.10

Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.7220149253731343}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.8022388059701492}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.8824626865671641}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 0.9626865671641791}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 1.042910447761194}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Sc

/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:48:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:48:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:48:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:48:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tys2/anaconda3/lib/python3.10

Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 1.2033582089552237}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 1.2835820895522387}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 1.3638059701492538}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 0.6791044776119404, 1: 1.4440298507462686}


...

/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:49:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:49:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:49:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:49:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tys2/anaconda3/lib/python3.10

Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 2.5671641791044775}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 2.6473880597014925}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 2.7276119402985075}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 2.807835820895522}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 2.888059701492537}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme f

/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:49:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:49:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:49:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:49:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tys2/anaconda3/lib/python3.10

Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 3.048507462686567}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 3.1287313432835817}
Saving Validation Results:
Saving Test Results:
--------------------------------------------------

Weighing Scheme for Training: 
{0: 6.791044776119404, 1: 3.2089552238805967}
Saving Validation Results:
Saving Test Results:


/Users/tys2/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:49:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "tree_method" } are not used.

  warnings.warn(smsg, UserWarning)


In [129]:
#save validation results
val_results = pd.DataFrame(val_dict)
val_results.to_csv(os.path.join(outdir, "XGBoost_ValResults.csv"), index = False)


#save test results
test_results = pd.DataFrame(test_dict)
test_results.to_csv(os.path.join(outdir, "XGBoost_TestResults.csv"), index = False)